In [ ]:
#| default_exp core

# Core

Useful functions and utilities for LibreGrid.

In [ ]:
#| export
import httpx
import os

In [ ]:
#| export
def get_smart_meter_data():
    "Python function to get smart meter data from Octopus Home Mini API"
    OE_API_KEY = os.getenv("OE_API_KEY")
    # TODO